In [5]:
pip install --upgrade albumentations


Note: you may need to restart the kernel to use updated packages.


In [6]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
import cv2
import numpy as np
import torch
import os
import json
import urllib.request
     

In [ ]:
import cv2
import numpy as np
import json
from PIL import Image

# Define the local file paths and annotations as lists
original_images = ["/kaggle/input/images/val2017-20241107T150103Z-001/val2017/000000000285.jpg"]
original_annotations = [
    '/kaggle/input/images/instances_val2017.json'
]

# Define the processing function
def process_data(imgs, anns):
    processed_imgs = []
    processed_anns = []

    for img_path, ann_path in zip(imgs, anns):
        # Read the image from the local path
        img = cv2.imread(img_path)

        # Check if the image is loaded successfully
        if img is None:
            print(f"Error loading image at {img_path}")
            continue

        # Load and parse JSON annotations
        try:
            with open(ann_path, 'r') as f:
                annotation_data = json.load(f)
        except Exception as e:
            print(f"Error loading annotation JSON: {e}")
            continue

        # Convert to grayscale
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Convert to binary using thresholding
        _, binary_img = cv2.threshold(
            gray_img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU
        )

        # Resize image
        resized_img = cv2.resize(img, (224, 224))

        # Rotate if height > width
        height, width, _ = img.shape
        if height > width:
            rotated_img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        else:
            rotated_img = img

        # Sharpen the image
        sharpened_img = cv2.GaussianBlur(rotated_img, (0, 0), sigmaX=1, sigmaY=1)
        sharpened_img = cv2.addWeighted(rotated_img, 1.5, sharpened_img, -0.5, 0)

        # Convert sharpened image to grayscale
        bw_img = cv2.cvtColor(sharpened_img, cv2.COLOR_BGR2GRAY)

        # Append all processed images and annotations
        processed_imgs.extend([img, binary_img, resized_img, rotated_img, sharpened_img, bw_img])
        processed_anns.extend([annotation_data] * 6)

    return processed_imgs, processed_anns

# Call the processing function
processed_imgs, processed_anns = process_data(original_images, original_annotations)

# Example placeholder for `download_and_display` function
def display_image_with_annotation(image, annotation):
    """
    Displays the processed image with annotations.
    """
    # Convert BGR to RGB for matplotlib display
    img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) if len(image.shape) == 3 else image

    # Display the image
    plt.imshow(img_rgb, cmap='gray' if len(image.shape) == 2 else None)
    plt.title(f"Annotation: {annotation}")
    plt.axis('off')
    plt.show()

# Display the processed images
for img, ann in zip(processed_imgs, processed_anns):
    display_image_with_annotation(img, ann)


ValueError: Image size of 12348609x411 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 640x480 with 1 Axes>

In [8]:
import json
import cv2
import urllib.request
import numpy as np
import matplotlib.pyplot as plt

# Load COCO-style JSON file
json_filename = '/kaggle/input/images/instances_val2017.json'
with open(json_filename, 'r') as f:
    data = json.load(f)

def download_and_display(image_url, annotations):
    """
    Downloads an image from a URL, overlays annotations (bounding boxes),
    and displays the image.
    """
    try:
        resp = urllib.request.urlopen(image_url)
        img_array = np.asarray(bytearray(resp.read()), dtype=np.uint8)
        img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
    except Exception as e:
        print(f"Error downloading image: {e}")
        return

    # Draw bounding boxes
    for ann in annotations:
        x, y, w, h = ann['bbox']
        cv2.rectangle(img, (int(x), int(y)), (int(x+w), int(y+h)), (255, 0, 0), 2)

    # Convert BGR to RGB for display
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

# Get the number of images to display
num_images = int(input("Enter the number of images to load (default is all): ") or len(data['images']))

for i, img_data in enumerate(data['images']):
    if i >= num_images:
        break

    # Retrieve image details
    img_id = img_data['id']
    img_url = img_data.get('coco_url', None)
    if not img_url:
        print(f"Image {img_id} has no URL.")
        continue

    # Get annotations for the image
    annotations = [
        ann for ann in data.get('annotations', [])
        if ann['image_id'] == img_id
    ]

    # Display the image with annotations
    download_and_display(img_url, annotations)


Enter the number of images to load (default is all):  10


Error downloading image: <urlopen error [Errno -3] Temporary failure in name resolution>
Error downloading image: <urlopen error [Errno -3] Temporary failure in name resolution>
Error downloading image: <urlopen error [Errno -3] Temporary failure in name resolution>
Error downloading image: <urlopen error [Errno -3] Temporary failure in name resolution>
Error downloading image: <urlopen error [Errno -3] Temporary failure in name resolution>
Error downloading image: <urlopen error [Errno -3] Temporary failure in name resolution>
Error downloading image: <urlopen error [Errno -3] Temporary failure in name resolution>
Error downloading image: <urlopen error [Errno -3] Temporary failure in name resolution>
Error downloading image: <urlopen error [Errno -3] Temporary failure in name resolution>
Error downloading image: <urlopen error [Errno -3] Temporary failure in name resolution>


In [12]:
import cv2
import numpy as np
from PIL import Image

def process_images(original_images, original_annotations):
    processed_images = []
    processed_annotations = []

    for img_path, annotation in zip(original_images, original_annotations):
        # Load the image from a local path
        img = cv2.imread(img_path)

        # Check if the image was loaded correctly
        if img is None:
            print(f"Error loading image at {img_path}")
            continue

        # Convert to grayscale
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Apply binary thresholding
        binary_img = cv2.threshold(gray_img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

        # Resize image
        resized_img = cv2.resize(img, (224, 224))

        # Rotate if height > width
        height, width, _ = img.shape
        if height > width:
            rotated_img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        else:
            rotated_img = img

        # Sharpen the image to reduce blurriness
        sharpened_img = cv2.GaussianBlur(rotated_img, (0, 0), sigmaX=1, sigmaY=1)
        sharpened_img = cv2.addWeighted(rotated_img, 1.5, sharpened_img, -0.5, 0)

        # Convert the sharpened image to grayscale (black and white)
        bw_img = cv2.cvtColor(sharpened_img, cv2.COLOR_BGR2GRAY)

        # Add processed images and annotations to lists
        processed_images.extend([img, binary_img, resized_img, rotated_img, sharpened_img, bw_img])
        processed_annotations.extend([annotation] * 6)

    return processed_images, processed_annotations

# Example image paths and annotations
original_images = ["/kaggle/input/images/val2017-20241107T150103Z-001/val2017/000000000632.jpg"]
original_annotations = ['Annotation for image 1']

# Process the images
processed_images, processed_annotations = process_images(original_images, original_annotations)

# Define a placeholder `download_and_display` function
def download_and_display(img_path, annotation, image):
    print(f"Displaying processed image for {img_path} with annotation: {annotation}")
    # Example display using OpenCV (uncomment if running in an environment that supports cv2.imshow)
    # cv2.imshow("Processed Image", image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

# Display the processed images
for i, (img, annotation) in enumerate(zip(processed_images, processed_annotations)):
    download_and_display(original_images[0], annotation, image=img)

Displaying processed image for /kaggle/input/images/val2017-20241107T150103Z-001/val2017/000000000632.jpg with annotation: Annotation for image 1
Displaying processed image for /kaggle/input/images/val2017-20241107T150103Z-001/val2017/000000000632.jpg with annotation: Annotation for image 1
Displaying processed image for /kaggle/input/images/val2017-20241107T150103Z-001/val2017/000000000632.jpg with annotation: Annotation for image 1
Displaying processed image for /kaggle/input/images/val2017-20241107T150103Z-001/val2017/000000000632.jpg with annotation: Annotation for image 1
Displaying processed image for /kaggle/input/images/val2017-20241107T150103Z-001/val2017/000000000632.jpg with annotation: Annotation for image 1
Displaying processed image for /kaggle/input/images/val2017-20241107T150103Z-001/val2017/000000000632.jpg with annotation: Annotation for image 1


In [ ]:
import os
import cv2
import torch
import numpy as np
from torch.utils.data import Dataset

class CocoDataset(Dataset):
    def __init__(self, images, annotations, category_map, image_dir, transform=None):
        """
        Custom dataset for COCO-like dataset.

        Args:
            images (list): List of images in the dataset.
            annotations (list): List of annotations for each image.
            category_map (dict): Mapping of category IDs to category names.
            image_dir (str): Directory path where images are stored.
            transform (callable, optional): Transformations to be applied to images and annotations.
        """
        self.images = images
        self.annotations = annotations
        self.category_map = category_map
        self.image_dir = image_dir
        self.transform = transform
        self.image_annotations = self._map_annotations_to_images()

    def _map_annotations_to_images(self):
        """
        Groups annotations by image ID.

        Returns:
            dict: Mapping from image IDs to their respective annotations.
        """
        annotation_map = {}
        for annotation in self.annotations:
            image_id = annotation['image_id']
            if image_id not in annotation_map:
                annotation_map[image_id] = []
            annotation_map[image_id].append(annotation)
        return annotation_map

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        """
        Returns an image and its annotations as a dictionary.

        Args:
            idx (int): Index of the image.

        Returns:
            tuple: (image, target) where image is the transformed image, and target is a dictionary containing bounding boxes and labels.
        """
        image_info = self.images[idx]
        img_path = os.path.join(self.image_dir, image_info['file_name'])
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Get annotations for the image
        image_id = image_info['id']
        annotations = self.image_annotations.get(image_id, [])

        boxes = []
        labels = []
        for ann in annotations:
            x, y, width, height = ann['bbox']
            boxes.append([x, y, x + width, y + height])
            labels.append(ann['category_id'])

        # Convert boxes and labels to numpy arrays
        boxes = np.array(boxes)
        labels = np.array(labels)

        # Apply transformations if any
        if self.transform:
            transformed = self.transform(image=image, bboxes=boxes, labels=labels)
            image = transformed['image']
            boxes = transformed['bboxes']
            labels = transformed['labels']

        # Convert to PyTorch tensors
        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)
        target = {"boxes": boxes, "labels": labels}

        return image, target

    def get_image_info(self, idx):
        """
        Returns metadata for a specific image.

        Args:
            idx (int): Index of the image.

        Returns:
            dict: Metadata for the image.
        """
        return self.images[idx]

    def visualize_image(self, idx):
        """
        Displays an image along with its bounding boxes.

        Args:
            idx (int): Index of the image.
        """
        import matplotlib.pyplot as plt
        import matplotlib.patches as patches

        image, target = self[idx]
        boxes = target['boxes'].numpy()
        labels = target['labels'].numpy()

        fig, ax = plt.subplots(1)
        ax.imshow(image)

        for box, label in zip(boxes, labels):
            x, y, x2, y2 = box
            width, height = x2 - x, y2 - y
            rect = patches.Rectangle((x, y), width, height, linewidth=2, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
            ax.text(x, y, self.category_map[label], color='white', verticalalignment='top', bbox=dict(facecolor='red', alpha=0.5))

        plt.axis('off')
        plt.show()

    def __repr__(self):
        return f"CocoDataset(num_images={len(self.images)}, num_annotations={len(self.annotations)}, image_dir='{self.image_dir}')"

In [18]:
import torch
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os

# COCO dataset class labels
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat',
    'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog',
    'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite',
    'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle',
    'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
    'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
    'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
    'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

def load_object_detection_model(weights_path=None):
    """
    Load Faster R-CNN model with local weights
    """
    try:
        # Initialize model architecture
        model = fasterrcnn_resnet50_fpn(pretrained=False, progress=False)
        
        # Load weights if path provided
        if weights_path and os.path.exists(weights_path):
            model.load_state_dict(torch.load(weights_path))
        else:
            print("Warning: No weights loaded. Model performance will be random.")
        
        model.eval()  # Set to evaluation mode
        return model
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

def preprocess_image(image_path):
    """
    Preprocess image for object detection
    """
    try:
        # Image transformation pipeline
        transform = transforms.Compose([
            transforms.ToTensor(),
        ])
        
        # Load and transform image
        img = Image.open(image_path).convert('RGB')
        img_tensor = transform(img).unsqueeze(0)  # Add batch dimension
        return img, img_tensor
    except Exception as e:
        print(f"Error preprocessing image: {e}")
        return None, None

def visualize_predictions(img, predictions, threshold=0.5):
    """
    Visualize object detection predictions
    """
    try:
        # Create figure and axes
        fig, ax = plt.subplots(1, figsize=(12, 9))
        ax.imshow(img)
        
        # Process predictions
        for box, label, score in zip(predictions['boxes'], predictions['labels'], predictions['scores']):
            if score > threshold:
                x1, y1, x2, y2 = box.int().tolist()
                label_name = COCO_INSTANCE_CATEGORY_NAMES[label]
                
                # Draw bounding box
                rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, 
                                         linewidth=2, edgecolor='red', facecolor='none')
                ax.add_patch(rect)
                
                # Add label and confidence
                ax.text(x1, y1-10, f'{label_name} {score:.2f}', 
                        color='white', fontsize=10,
                        bbox=dict(facecolor='red', alpha=0.5))
        
        plt.axis('off')
        plt.tight_layout()
        plt.show()
    except Exception as e:
        print(f"Visualization error: {e}")

def main(image_path, weights_path=None):
    """
    Main object detection pipeline
    """
    # Load model
    model = load_object_detection_model(weights_path)
    if not model:
        return
    
    # Preprocess image
    orig_img, img_tensor = preprocess_image(image_path)
    if orig_img is None or img_tensor is None:
        return
    
    # Perform detection
    with torch.no_grad():
        predictions = model(img_tensor)[0]
    
    # Visualize results
    visualize_predictions(orig_img, predictions)

# Example usage
if __name__ == "__main__":
    # Update these paths
    image_path = "/kaggle/input/images/val2017-20241107T150103Z-001/val2017/000000000776.jpg"
    weights_path = "/kaggle/input/fasterrcnn/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth"
    main(image_path, weights_path)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


Error loading model: <urlopen error [Errno -3] Temporary failure in name resolution>
